In [36]:
import pandas as pd
import numpy as np
import seaborn as sns


In [37]:
df = sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [6]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [7]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [38]:
from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()

In [39]:
df['time']=Le.fit_transform(df['time'])

In [40]:
df.sample(5)

,total_bill,tip,sex,smoker,day,time,size
131,20.27,2.83,Female,No,Thur,1,2
215,12.90,1.10,Female,Yes,Sat,0,2
4,24.59,3.61,Female,No,Sun,0,4
119,24.08,2.92,Female,No,Thur,1,4
179,34.63,3.55,Male,Yes,Sun,0,2


In [41]:
df.time.unique()

array([0, 1])

In [42]:
X=df.drop(columns=['time'])
Y =df['time']

In [43]:
from sklearn.model_selection  import train_test_split
X_train , X_test , Y_train ,Y_test =train_test_split(X,Y , random_state=40 , test_size=0.20)

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [55]:
categorical_col =['smoker','sex','day']
numerical_col = ['total_bill' , 'tip' ,'size']

In [56]:
num_pipeline =Pipeline(
    steps=[
        ('imputer' , SimpleImputer(strategy='median')),
        ('scaler' , StandardScaler())
         ]

)

In [57]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [58]:
cat_pipeline =Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('Onehotencoder', OneHotEncoder())
    ]
)

In [59]:
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('Onehotencoder', OneHotEncoder())])

In [60]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_col),
    ('cat_pipeline', cat_pipeline,categorical_col)   
])

In [61]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Onehotencoder',
                                                  OneHotEncoder())]),
                                 ['smoker', 'sex', 'day'])])

In [62]:
X_train=preprocessor.fit_transform(X_train)


In [63]:
X_test =preprocessor.fit_transform(X_test)

In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [92]:
models={
    'Random_forest':RandomForestClassifier(),
    'Logistic_regression':LogisticRegression(),
    'Decision_tree':DecisionTreeClassifier()   
}

In [93]:
from sklearn.metrics import accuracy_score

In [98]:
def evaluate_model(X_train , Y_train , X_test, Y_test , models):
    report={}
    for i in range(len(models)):
        model= list(models.values())[i]
        model.fit(X_train , Y_train)
        
        
        #predict testing data
        y_test_pred = model.predict(X_test)\
        
        #GEt accuracy for test data prediction
        
        test_model_score =accuracy_score(Y_test , y_test_pred)
        
        report[list(models.keys())[i]]= test_model_score
         
            
            
            
    return report
    

In [99]:
evaluate_model(X_train, Y_train, X_test, Y_test, models)

{'Random_forest': 0.9591836734693877,
 'Logistic_regression': 0.9591836734693877,
 'Decision_tree': 0.9387755102040817}

In [73]:
Classifier=RandomForestClassifier()

In [87]:
#hyper parameter tuning
params ={'max_depth': [3,5,10,None],
        'n_estimators':[100,200,300],
        'criterion':['gini' ,'entropy']}

In [76]:
from sklearn.model_selection import RandomizedSearchCV

In [78]:
cv=RandomizedSearchCV(Classifier, param_distributions=params,scoring='accuracy' , cv=5 , verbose=3)
cv.fit(X_train,Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=1.000 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.974 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.949 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.949 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=100;, score=1.000 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.949 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.949 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [80]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 5, 'criterion': 'entropy'}